In [31]:
## NE 155 Final Analysis Project
# Ali Albaaj

# reading labview data converted into csv file, pulling out the data we want

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from scipy.interpolate import interp1d
from scipy import stats

pulse = pd.read_csv('pulse.csv')

def readCSVColumn(dataList):
    
    dataList = np.asarray(dataList)
    place = 0
    arbitrary = 0
    for i in range(len(dataList)):

        try:
            if i == (31 * arbitrary + 19):
                arbitrary += 1
                continue
            if np.float(dataList[i]) == 24:
                continue
            elif np.float(dataList[i]) == 20:
                continue
            elif np.float(dataList[i]) == 2:
                continue
            elif np.float(dataList[i]) == 0.05:
                continue
            else:
                dataList[place] = np.float(dataList[i])
                place += 1
        except:
            continue

    dataList = dataList[0:place]
    dataList = [value for value in dataList if not math.isnan(value)]
    dataList = np.asarray(dataList)
    
    return dataList

inline2 = readCSVColumn(pulse['Unnamed: 2'])
pebble3 = readCSVColumn(pulse['Unnamed: 3'])
pebble5 = readCSVColumn(pulse['Unnamed: 5'])
pebble8 = readCSVColumn(pulse['Unnamed: 8'])
inline11 = readCSVColumn(pulse['Unnamed: 11'])

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import lvm_read

# Differentiation Coding


def TemperatureDiff(tempVector, timeVector, method, order):
    
    length = len(tempVector)
    h = timeVector[1] - timeVector[0]
    diffVector = np.zeros(len(tempVector))
    i=1
    
    if method == 'center':
        if order == 2:
            
            while i < length-1:
                diffVector[i] = ((tempVector[i+1] - tempVector[i-1]) / (2*h))
                i += 1
        
        elif order == 4:
            
            i = 2
            while i < length-2:
                diffVector[i] = (tempVector[i-2] - 8*tempVector[i-1] + 8*tempVector[i+1] 
                                - tempVector[i+2]) / (2*h)
                i += 1
                
        else:
            return print('choose integer 2 or 4')
            
    elif method == 'forward':
        if order == 1:
            
            while i < length-1:
                diffVector[i] = ((tempVector[i+1] - tempVector[i]) / (2*h))
                i += 1
        
        elif order == 2:
            
            while i < length-2:
                diffVector[i] = (-3*tempVector[i] + 4*tempVector[i+1] - tempVector[i+1]) / (2*h)
                i += 1
                
        else:
            return print('choose integer 1 or 2')      
        
    elif method == 'backward':
        if order == 1:
            
            while i < length-1:
                diffVector[i] = ((tempVector[i] - tempVector[i-1]) / (2*h))
                i += 1
                
        elif order == 2:
            
            while i < length-2:
                diffVector[i] = (3*tempVector[i] - 4*tempVector[i-1] + tempVector[i-2]) / (2*h)
                i += 1
        else:
            return print('choose integer 1 or 2')
        
    else:
        return print('function method not defined. Please choose "center", "forward", or "backward"')
    
    return diffVector



In [33]:
# Interpolating code
import operator

def FindPoly(xData, yData):
    
    xarray = np.zeros((len(xData), len(xData)))
    
    for i in range(len(xData)):
        m = 0
        for m in range(len(yData)):
            xarray[i,m] = np.power(xData[i], len(xData)-m)
            m +=1  
    poly = np.dot(np.linalg.inv(xarray), yData)
    poly = poly.T
    return poly

def PolyVal(coefficients, xData):
    
    n = len(coefficients)
    yEvaluated = np.copy(xData)
    
    for h in range(len(xData)):
        equation = 0
        
        for i in range(n):
            equation = coefficients[i] * np.power(xData[h], n-i) + equation
        yEvaluated[h] = equation
    
    return yEvaluated


# code verification
xPt = np.array([-2, 1, -4, 5, 2])
yPt = np.array([2, 2.7, -7, 1, 2])
x = np.arange(-4, 5, 0.1)
poly = FindPoly(xPt, yPt)
x = np.arange(-4, 5, 0.1)
y = PolyVal(poly, x)
cubic = interp1d(xPt, yPt, 'cubic')

plt.plot(x, y, 'b', label='Custom Polynomial Interpolation')
plt.plot(xPt, yPt, 'o')
plt.plot(x, cubic(x), 'r', label='Cubic Spline Interpolation')
plt.title('Comparison of interpolation codes')
plt.legend(loc='lower left')
plt.show()

In [34]:
## Graphing temperatures

time = np.arange(44320) * 0.05
plt.plot(time, inline2, 'r', label='inline2')
plt.plot(time, inline11, 'g', label='inline11')
plt.plot(time, pebble3, 'y', label='pebble3')
plt.plot(time, pebble5, 'b', label='pebble5')
plt.plot(time, pebble8, 'k', label='pebble8')
plt.legend(loc='upper left')
plt.xlabel('time(s)')
plt.ylabel('temperature (C)')
plt.title('Fluid bulk and pebble core temperature (pulse test)')
plt.show()

In [35]:
# pebble temperatures

plt.plot(time, pebble3, 'y', label='pebble3')
plt.plot(time, pebble5, 'b', label='pebble5')
plt.plot(time, pebble8, 'r', label='pebble8')
plt.legend(loc='upper left')
plt.xlabel('time(s)')
plt.ylabel('temperature (C)')
plt.title('pebble core temperature (pulse test)')
plt.show()

In [36]:
# plotting temperature and derivative, accuracy 2

pebble5Diff = TemperatureDiff(pebble5, x, 'center', 2)
inline11Diff = TemperatureDiff(inline11, x, 'center', 2)
plt.plot(time, pebble5, 'r', label='pebble temperature')
plt.plot(time, pebble5Diff, 'g', label='pebble derivative')
plt.xlabel('time(s)')
plt.ylabel('temperature(C)')
plt.legend(loc='upper left')
plt.title('pebble core temperature vs time pulse')
plt.show()

In [37]:
# plotting temperature vs derivative, accuracy 4

plt.plot(time, pebble3, 'r', label='pebble core temperature')
plt.plot(time, tempDer2, 'g', label='pebble core derivative')
plt.legend(loc='upper left')
plt.xlabel('time(s)')
plt.ylabel('temperature(C)')
plt.title('pebble temperature vs pebble core derivative')
plt.show()

In [38]:
# determining the heat transfer coefficient, acccuracy 2

Cp = 0.39
row = 8960
minusEpsilon = .52
Av = 567
tempDer = TemperatureDiff(pebble3, time, 'center', 2)
tempDer2 = TemperatureDiff(pebble3, time, 'center', 4)
deltaT = pebble3 - inline2
h = np.zeros(len(time))
for i in range(len(time)):
    h[i] = ((Cp*row*minusEpsilon)/Av) * abs(tempDer[i]/deltaT[i])

# removing outliers to give more clear graphs

h=np.delete(h, [29847, 29848, 29859])
print(np.median(h))
plt.plot(np.delete(time,[29847, 29848, 29859]), h, 'b', label='heat transfer coefficient')
plt.plot(time, pebble3, 'r', label='pebble core temperature')
plt.plot(time, inline2, 'g', label='bulk fluid temperature')
plt.legend(loc='upper left')
plt.xlabel('time(s)')
plt.ylabel('temperature(C)')
plt.title('Heat transfer coefficient vs pebble and bulk temperature')
plt.show()

0.960046451236


In [39]:
# determining the heat transfer coefficient, accuracy 4

h = np.zeros(len(time))
for i in range(len(time)):
    h[i] = ((Cp*row*minusEpsilon)/Av) * abs(tempDer2[i]/deltaT[i])

# removing outliers to give more clear graphs

h=np.delete(h, [29847, 29848, 29859])
print(np.median(h))
plt.plot(np.delete(time,[29847, 29848, 29859]), h, 'b', label='heat transfer coefficient')
plt.plot(time, pebble3, 'r', label='pebble core temperature')
plt.plot(time, inline2, 'g', label='bulk fluid temperature')
plt.legend(loc='upper left')
plt.xlabel('time(s)')
plt.ylabel('temperature(C)')
plt.title('Heat transfer coefficient vs pebble and bulk temperature')
plt.show()

7.70197564128


In [40]:
# cubic spline interpolation of limited data

xSpaced = time[0:44320:150]
newX = np.arange(4425) * 0.5
pebble5Spaced = pebble5[0:44320:150]
cubicPebble5 = interp1d(xSpaced, pebble5Spaced)
plt.plot(xSpaced, pebble5Spaced, 'o')
plt.plot(newX, cubicPebble5(newX), 'b', label='cubic spline interp')
plt.plot(time, pebble5, 'r', label='actual data')
plt.xlabel('time(s)')
plt.ylabel('temperature(C)')
plt.legend(loc='upper left')
plt.title('Spline with less data vs actual data')
plt.show()